<a href="https://colab.research.google.com/github/afrokyss/Africa-Covid-Report/blob/main/covid_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px

## Drop col



In [2]:
def drop_col(input_data):
  output_data = input_data.drop(columns = ['Province/State'])
  return output_data

## Rename Cols

In [3]:
def rename_col(input_data):
  output_data = input_data.rename(
      columns = {
          'Country/Region' : 'country',
          'Long': 'long',
          'Lat': 'lat'
      }
  )
  return output_data

## Melt dataframe

In [4]:
def covid_melt(input_data, value_val_name):
  output_data = input_data.melt(id_vars = ['lat', 'long', 'country'],
                                var_name = 'date_raw',
                                value_name = value_val_name)
  return output_data
  

## define date

In [5]:
def covid_date(input_data):
  output_data = input_data.assign(date = pd.to_datetime(input_data['date_raw'], format= '%m/%d/%y'))
  output_data.drop(columns = ['date_raw'], inplace = True)
  return output_data

## Rearange data 

In [6]:
def rearange_data(input_data, value_val_name):
  output_data = input_data[['country', 'date', 'lat', 'long', value_val_name]] 
  return output_data

## Compute data

In [7]:
def get_data(input_url, value_val_name):
  data = pd.read_csv(input_url)
  data = drop_col(data)
  data = rename_col(data)
  data = covid_melt(data, value_val_name)
  data = covid_date(data)
  data = rearange_data(data, value_val_name)
  
  return data


## Gete data

In [8]:
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv' 

confirmed_cases = get_data(url_confirmed, 'confirmed')
deaths_cases = get_data(url_deaths, 'deaths')
recovered_cases = get_data(url_recovered, 'recovered')

## merge dataframes

In [9]:
deaths_cases.drop(columns = ['lat','long'], inplace = True)
recovered_cases.drop(columns = ['lat','long'], inplace = True)

In [10]:
df_covid = (confirmed_cases
            .merge(deaths_cases, on=['country', 'date'], how='left')
            .merge(recovered_cases, on=['country', 'date'], how='left')
            )

In [11]:
df_covid

,country,date,lat,long,confirmed,deaths,recovered
0,Afghanistan,2020-01-22,33.939110,67.709953,0,0,0
1,Albania,2020-01-22,41.153300,20.168300,0,0,0
2,Algeria,2020-01-22,28.033900,1.659600,0,0,0
3,Andorra,2020-01-22,42.506300,1.521800,0,0,0
4,Angola,2020-01-22,-11.202700,17.873900,0,0,0
...,...,...,...,...,...,...,...
10504860,West Bank and Gaza,2020-10-12,31.952200,35.233200,44684,387,38228
10504861,Western Sahara,2020-10-12,24.215500,-12.885800,10,1,8
10504862,Yemen,2020-10-12,15.552727,48.516388,2052,596,1329
10504863,Zambia,2020-10-12,-13.133897,27.849332,15549,345,14682


In [12]:
df_covid[df_covid['country']=='Egypt']

,country,date,lat,long,confirmed,deaths,recovered
36721,Egypt,2020-01-22,26.820553,30.802498,0,0,0
76362,Egypt,2020-01-23,26.820553,30.802498,0,0,0
116003,Egypt,2020-01-24,26.820553,30.802498,0,0,0
155644,Egypt,2020-01-25,26.820553,30.802498,0,0,0
195285,Egypt,2020-01-26,26.820553,30.802498,0,0,0
...,...,...,...,...,...,...,...
10343381,Egypt,2020-10-08,26.820553,30.802498,104156,6017,97524
10383022,Egypt,2020-10-09,26.820553,30.802498,104262,6029,97592
10422663,Egypt,2020-10-10,26.820553,30.802498,104387,6040,97643
10462304,Egypt,2020-10-11,26.820553,30.802498,104516,6052,97688


## Add active cases

In [13]:
df_covid['active']= df_covid['confirmed']-df_covid['deaths']-df_covid["recovered"]

## Add new confirmed, deaths and new recovered cases

In [14]:
new_cases_all = df_covid.sort_values(by = ['country', 'date'])

In [15]:
new_cases_all = new_cases_all.filter(['country', 'date', 'confirmed', 'deaths', 'recovered'])

In [16]:
new_cases_all = new_cases_all.groupby(['country'])

In [17]:
new_cases_all = new_cases_all[['confirmed', 'deaths', 'recovered']].diff()

In [18]:
new_cases_all = new_cases_all.rename(columns={'confirmed':'new_confirmed', 'deaths':'new_deaths', 'recovered': 'new_recovered'})

In [19]:
df_covid =  df_covid.join(new_cases_all, how='left')


## Keep only African countries/join countries and regions

In [20]:
# replace some countries names
df_covid['country'] = df_covid['country'].replace('Cabo Verde', 'Cape Verde')
df_covid['country'] = df_covid['country'].replace('Congo (Brazzaville)', 'Congo')
df_covid['country'] = df_covid['country'].replace('Congo (Kinshasa)', 'Democratic Republic of Congo')
df_covid['country'] = df_covid['country'].replace('Eswatini', 'Swaziland')

In [21]:
region_country = pd.read_csv('/content/drive/My Drive/covid_africa_data/african_regions.csv')

In [22]:
list_country = region_country['location'].tolist()

In [23]:
df_covid= df_covid.loc[df_covid['country'].isin(list_country).dropna()]

In [24]:
region_country.rename(columns={'location': 'country'}, inplace=True)

In [25]:
df_covid = df_covid.merge(region_country, on=['country'], how='inner')

In [26]:
old_columns = [x for x in df_covid.columns if x not in ['region']]
insert_index = old_columns.index('country') + 1
df_covid = df_covid[old_columns[:insert_index] + ['region'] + old_columns[insert_index:]]


In [27]:
df_covid[df_covid['country']=='Egypt']

,country,region,date,lat,long,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered
3975,Egypt,North,2020-01-22,26.820553,30.802498,0,0,0,0,NaN,NaN,NaN
3976,Egypt,North,2020-01-23,26.820553,30.802498,0,0,0,0,0.0,0.0,0.0
3977,Egypt,North,2020-01-24,26.820553,30.802498,0,0,0,0,0.0,0.0,0.0
3978,Egypt,North,2020-01-25,26.820553,30.802498,0,0,0,0,0.0,0.0,0.0
3979,Egypt,North,2020-01-26,26.820553,30.802498,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4235,Egypt,North,2020-10-08,26.820553,30.802498,104156,6017,97524,615,121.0,7.0,32.0
4236,Egypt,North,2020-10-09,26.820553,30.802498,104262,6029,97592,641,106.0,12.0,68.0
4237,Egypt,North,2020-10-10,26.820553,30.802498,104387,6040,97643,704,125.0,11.0,51.0
4238,Egypt,North,2020-10-11,26.820553,30.802498,104516,6052,97688,776,129.0,12.0,45.0


In [28]:
df_covid.to_csv('african_covid_report.csv', index_label=False)

In [29]:
df_covid.to_csv('/content/drive/My Drive/african_covid_report.csv', index_label=False)

In [30]:
df = pd.read_csv('/content/african_covid_report.csv')

In [31]:
country_code = pd.read_csv('countrycode_data.csv')


In [32]:
country_code = country_code[country_code['continent']=='Africa']
country_code['country_name']=country_code['country_name'].str.title()
country_code['country_name'] =country_code['country_name'].replace("Cote D'Ivoire","Cote d'Ivoire")
country_code['country_name']=country_code['country_name'].replace('Congo, The Democratic Republic Of','Democratic Republic of Congo')
country_code['country_name']=country_code['country_name'].replace('Congo, Republic Of','Congo')
country_code['country_name']=country_code['country_name'].replace('Libyan Arab Jamahiriya','Libya')
country_code['country_name']=country_code['country_name'].replace('Sao Tome And Principe', 'Sao Tome and Principe')
country_code['country_name']=country_code['country_name'].replace('Tanzania, United Republic Of', 'Tanzania')



In [33]:
country_code=country_code.rename(columns={'country_name':'country'})

In [34]:
country_code = country_code.sort_values('country', ascending=True)


In [35]:
df= pd.merge(df, country_code, how='left', on=['country'])

In [36]:
df.head()

,country,region_x,date,lat,long,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered,cowc,cown,fao,fips104,imf,iso2c,iso3c,iso3n,un,wb,continent,region_y,regex
0,Algeria,North,2020-01-22,28.0339,1.6596,0,0,0,0,NaN,NaN,NaN,ALG,615.0,4.0,AG,612.0,DZ,DZA,12.0,12.0,DZA,Africa,Northern Africa,.*algeria.*
1,Algeria,North,2020-01-23,28.0339,1.6596,0,0,0,0,0.0,0.0,0.0,ALG,615.0,4.0,AG,612.0,DZ,DZA,12.0,12.0,DZA,Africa,Northern Africa,.*algeria.*
2,Algeria,North,2020-01-24,28.0339,1.6596,0,0,0,0,0.0,0.0,0.0,ALG,615.0,4.0,AG,612.0,DZ,DZA,12.0,12.0,DZA,Africa,Northern Africa,.*algeria.*
3,Algeria,North,2020-01-25,28.0339,1.6596,0,0,0,0,0.0,0.0,0.0,ALG,615.0,4.0,AG,612.0,DZ,DZA,12.0,12.0,DZA,Africa,Northern Africa,.*algeria.*
4,Algeria,North,2020-01-26,28.0339,1.6596,0,0,0,0,0.0,0.0,0.0,ALG,615.0,4.0,AG,612.0,DZ,DZA,12.0,12.0,DZA,Africa,Northern Africa,.*algeria.*


In [37]:
df = df.drop(columns=['region_y', 'cowc','cown', 'fao','fips104','imf','iso2c', 'iso3n','un','wb','continent', 'regex'])

In [38]:
df

,country,region_x,date,lat,long,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered,iso3c
0,Algeria,North,2020-01-22,28.033900,1.659600,0,0,0,0,NaN,NaN,NaN,DZA
1,Algeria,North,2020-01-23,28.033900,1.659600,0,0,0,0,0.0,0.0,0.0,DZA
2,Algeria,North,2020-01-24,28.033900,1.659600,0,0,0,0,0.0,0.0,0.0,DZA
3,Algeria,North,2020-01-25,28.033900,1.659600,0,0,0,0,0.0,0.0,0.0,DZA
4,Algeria,North,2020-01-26,28.033900,1.659600,0,0,0,0,0.0,0.0,0.0,DZA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14570,Zimbabwe,Southern,2020-10-08,-19.015438,29.154857,7951,229,6446,1276,32.0,0.0,5.0,ZWE
14571,Zimbabwe,Southern,2020-10-09,-19.015438,29.154857,7994,229,6474,1291,43.0,0.0,28.0,ZWE
14572,Zimbabwe,Southern,2020-10-10,-19.015438,29.154857,8010,230,6492,1288,16.0,1.0,18.0,ZWE
14573,Zimbabwe,Southern,2020-10-11,-19.015438,29.154857,8011,230,6504,1277,1.0,0.0,12.0,ZWE


In [39]:
df = df.rename(columns={'region_x': 'region', 'iso3c': 'iso_alpha'})


In [40]:
df = df[['country'] + ['iso_alpha', 'region'] + ['date', 'lat', 'long', 'confirmed', 'deaths', 'recovered', 'active', 'new_confirmed', 'new_deaths', 'new_recovered']]


In [41]:
df.to_csv('african_covid_report_updated.csv', index_label=False)

In [42]:
df.to_csv('african_covid_report_last.csv', index_label = False)

In [43]:
df[df['country']=='Egypt']

,country,iso_alpha,region,date,lat,long,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered
3975,Egypt,EGY,North,2020-01-22,26.820553,30.802498,0,0,0,0,NaN,NaN,NaN
3976,Egypt,EGY,North,2020-01-23,26.820553,30.802498,0,0,0,0,0.0,0.0,0.0
3977,Egypt,EGY,North,2020-01-24,26.820553,30.802498,0,0,0,0,0.0,0.0,0.0
3978,Egypt,EGY,North,2020-01-25,26.820553,30.802498,0,0,0,0,0.0,0.0,0.0
3979,Egypt,EGY,North,2020-01-26,26.820553,30.802498,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,Egypt,EGY,North,2020-10-08,26.820553,30.802498,104156,6017,97524,615,121.0,7.0,32.0
4236,Egypt,EGY,North,2020-10-09,26.820553,30.802498,104262,6029,97592,641,106.0,12.0,68.0
4237,Egypt,EGY,North,2020-10-10,26.820553,30.802498,104387,6040,97643,704,125.0,11.0,51.0
4238,Egypt,EGY,North,2020-10-11,26.820553,30.802498,104516,6052,97688,776,129.0,12.0,45.0


## Insert country pop

In [44]:
df_pop = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')

In [45]:
df_pop.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2019-12-31,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-01-01,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-01-02,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-01-03,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-01-04,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498


In [46]:
df_pop_africa = df_pop[df_pop['continent']=='Africa']

In [47]:
df_pop_africa = df_pop_africa.rename(columns={'location':'country'})

In [48]:
df_pop_africa = df_pop_africa[['country', 'population']]

In [49]:
df_pop_africa = df_pop_africa.sort_values(by=['country'], ascending=True)

In [50]:
df_test = df.merge(df_pop_africa, on='country', how='left').drop_duplicates()

In [51]:
df_test.to_csv('african_covid_report_last.csv', index_label=False)

In [52]:
#ig= px.scatter(df_test, x = 'confirmed', y = 'deaths', animation_frame='date', animation_group='country',
                #size='confirmed', hover_name='country', log_x=True, log_y=True, size_max=45, 
                #range_x=[100, 1200000], range_y=[25, 40000], color = 'country', hover_data=['recovered', 'date'],
                #text='country', title='Confirmed cases vs deaths cases in Africa')

#fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration']=50


#fig.show()

In [53]:
#fig = px.line(df, x='country', y='confirmed', color='country', 
              #animation_frame='date', animation_group='country')
#fig.show()